<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Suppose we wish to perform a median filter with even window size on a ``float16``
image. Furthermore, the image has NaNs that we wish to be ignored (and effectively
removed by the filter). `median_filter` does not support ``float16`` data, its
behavior when NaNs are present is not defined, and for even window sizes, it does
not return the usual sample median - the average of the two middle elements. This
would be an excellent use case for `vectorized_filter` with
``function=np.nanmedian``, which supports the required interface: it accepts a
data array of any shape as the first positional argument, and tuple of axes as
keyword argument ``axis``.


In [ ]:
import numpy as np
from scipy import datasets, ndimage
from scipy.ndimage import vectorized_filter
import matplotlib.pyplot as plt
ascent = ndimage.zoom(datasets.ascent(), 0.5).astype(np.float16)
ascent[::16, ::16] = np.nan
result = vectorized_filter(ascent, function=np.nanmedian, size=4)

Plot the original and filtered images.


In [ ]:
fig = plt.figure()
plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ax1.imshow(ascent)
ax2.imshow(result)
fig.tight_layout()
plt.show()

Another need satisfied by `vectorized_filter` is to perform multi-output
filters. For instance, suppose we wish to filter an image according to the 25th
and 75th percentiles in addition to the median. We could perform the three
filters separately.


In [ ]:
ascent = ndimage.zoom(datasets.ascent(), 0.5)
def get_quantile_fun(p):
    return lambda x, axis: np.quantile(x, p, axis=axis)
ref1 = vectorized_filter(ascent, get_quantile_fun(0.25), size=4)
ref2 = vectorized_filter(ascent, get_quantile_fun(0.50), size=4)
ref3 = vectorized_filter(ascent, get_quantile_fun(0.75), size=4)
ref = np.stack([ref1, ref2, ref3])

However, `vectorized_filter` also supports filters that return multiple outputs
as long as `output` is unspecified and `batch_memory` is sufficiently high to
perform the calculation in a single chunk.


In [ ]:
def quartiles(x, axis):
    return np.quantile(x, [0.25, 0.50, 0.75], axis=axis)
res = vectorized_filter(ascent, quartiles, size=4, batch_memory=np.inf)
np.all(np.isclose(res, ref))

np.True_

The placement of the additional dimension(s) corresponding with multiple outputs
is at the discretion of `function`. `quartiles` happens to prepend one dimension
corresponding with the three outputs simply because that is the behavior of
`np.quantile`:


In [ ]:
res.shape == (3,) + ascent.shape

True

If we wished for this dimension to be appended:


In [ ]:
def quartiles(x, axis):
    return np.moveaxis(np.quantile(x, [0.25, 0.50, 0.75], axis=axis), 0, -1)
res = vectorized_filter(ascent, quartiles, size=4, batch_memory=np.inf)
res.shape == ascent.shape + (3,)

True

Suppose we wish to implment a "mode" filter - a filter that selects the most
frequently occuring value within the window. A simple (but rather slow)
approach is to use `generic_filter` with `scipy.stats.mode`.


In [ ]:
from scipy import stats
rng = np.random.default_rng()
input = rng.integers(255, size=(50, 50)).astype(np.uint8)
def simple_mode(input):
    return stats.mode(input, axis=None).mode
ref = ndimage.generic_filter(input, simple_mode, size=5)

If speed is important, `vectorized_filter` can take advantage of the performance
benefit of a vectorized callable.


In [ ]:
def vectorized_mode(x, axis=(-1,)):
    n_axes = 1 if np.isscalar(axis) else len(axis)
    x = np.moveaxis(x, axis, tuple(range(-n_axes, 0)))
    x = np.reshape(x, x.shape[:-n_axes] + (-1,))
    y = np.sort(x, axis=-1)
    i = np.concatenate([np.ones(y.shape[:-1] + (1,), dtype=bool),
                        y[..., :-1] != y[..., 1:]], axis=-1)
    indices = np.arange(y.size)[i.ravel()]
    counts = np.diff(indices, append=y.size)
    counts = np.reshape(np.repeat(counts, counts), y.shape)
    k = np.argmax(counts, axis=-1, keepdims=True)
    return np.take_along_axis(y, k, axis=-1)[..., 0]
res = vectorized_filter(input, vectorized_mode, size=5)
np.all(res == ref)

np.True_

Depending on the machine, the `vectorized_filter` version may be as much as
100x faster.
